In [1]:
import json
import pandas as pd
import numpy as np
from pprint import pprint
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import matplotlib.pyplot as plt
import string
import networkx as nx
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import plotly.figure_factory as ff
from scipy.cluster import hierarchy
#import seaborn as sns
#sns.set()
#stopwords = stopwords.words('english')

C:\Hyapp\Anaconda3-2019.3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Hyapp\Anaconda3-2019.3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Hyapp\Anaconda3-2019.3\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
# This function creates a pairwise similarity of all the windows commands using TFIDF vectors of commands description text.
# At the end we get a dataframe (matrix of nXn) of 685 commands where each commands' description is compared with all other 
# commands and a similarity score is returned. All the diagonal will have a value of 1. 
def create_pairwise_similarity(data):
    corpus, columns = [], [] 
    for d in range(len(data['commands'])):
        corpus.append(data['commands'][d]['description'])
    vect = TfidfVectorizer(min_df=1, stop_words="english" )                                                                                                                                                                                                 
    tfidf = vect.fit_transform(corpus)                                                                                                                                                                                                                       
    similarity_matrix = cosine_similarity(tfidf)
    for d in range(len(data['commands'])):
        columns.append(data['commands'][d]['commands'])
    sim_df = pd.DataFrame(similarity_matrix, columns=columns, index=columns)
    return sim_df


# The above created matrix is passed in this function with a threshold value, and we will get a dataframe and a dictionary
# of only those commands which have at least one other command with similarity score greater than or equal to threshold. 
# for example 'attributes' command have a similarity score of 0.944 with 'attributes volume', so these two commands will be in
# the dataframe along their similarity score.

def get_max_sim_comds(sim_df, threshold):
    cols = list(sim_df.columns)
    simi = dict()
    for c in cols:
        vv = sim_df.loc[sim_df[c]>threshold][[c]].index.values
        simi[c] = []
        for v in vv:  
            if v!=c and v not in simi.keys():  
                simi[c].append(v)
    new_sim = {k: v for k, v in simi.items() if len(simi[k])>0}
    data_items = new_sim.items()
    data_list = list(data_items)
    
    df = pd.DataFrame(data_list, columns=['comm1', 'comm2'])
    lst_col = 'comm2'
    
    r = pd.DataFrame({col:np.repeat(df[col].values, df[lst_col].str.len())
      for col in df.columns.drop(lst_col)}).assign(**{lst_col:np.concatenate(df[lst_col].values)})[df.columns]
    r['sim_score'] = ''
    
    for i in range(len(r['comm1'])):
        first_comm = r.loc[i][0]
        second_comm = r.loc[i][1]
        sim_val = sim_df.loc[[first_comm]][second_comm][0]
        r.loc[i, 'sim_score'] =  round(sim_val,3)
    return r, new_sim


# Given the dataframe created in the above function with 'comm1', 'comm2', 'sim_score' as three columns, we create a matrix of 
# all these commands in the function below. This function will return a dataframe (matrix) of the commands which have at least
# one command with high similarity (sim_Score>=threshold). We will use this for heatmaps.

def create_matrix_high_sim_comds(r):
    cols = np.unique(r['comm1'].values)
    indx = np.unique(r['comm2'].values)

    new_df = pd.DataFrame(columns=cols, index=indx)
    new_df = new_df.loc[:,~new_df.columns.duplicated()]

    new_df = new_df.loc[~new_df.index.duplicated(keep='first')]
    new_cols = new_df.columns
    new_indx = new_df.index
    
    for col in new_cols:
        for indx in new_indx:
            val = round(sim_df.loc[[col]][indx][0],3)
            new_df.loc[indx, col] = val
    return new_df

In [4]:
# reading the cleaned web scraped data of windows manuals
with open("./data/windows/windows_data.txt", "r") as read_file:
    data = json.load(read_file)

In [5]:
data['commands'][0]

{'commands': 'active',
 'description': 'On basic disks, marks the partition with focus as active. Only partitions can be marked as active. A partition must be selected for this operation to succeed. Use the\xa0select partition\xa0command to select a partition and shift the focus to it.  DiskPart only informs the basic input/output system (BIOS) or Extensible Firmware Interface (EFI) that the partition or volume is a valid system partition or system volume, and is capable of containing the operating system startup files. DiskPart does not check the contents of the partition. If you mistakenly mark a partition as active and it does not contain the operating system startup files, your computer might not start.',
 'syntax': ['active'],
 'parameters': {'None': 'None'}}

In [14]:

sim_df = create_pairwise_similarity(data)

In [15]:
sim_df

,active,add,add alias,add volume,append,arp,assign,assoc,at,atmadm,...,where,whoami,winnt32,winrs,winsat mem,winsat mfmedia,wmic,writer,wscript,xcopy
active,1.000000,0.086865,0.066447,0.130217,0.098375,0.053890,0.324707,0.097874,0.113317,0.066818,...,0.067725,0.101979,0.047571,0.028399,0.047851,0.030104,0.004136,0.059803,0.030928,0.046405
add,0.086865,1.000000,0.611041,0.503426,0.074089,0.053052,0.121681,0.055616,0.090894,0.050172,...,0.046931,0.078528,0.026676,0.030448,0.034164,0.033251,0.000000,0.055559,0.069300,0.007299
add alias,0.066447,0.611041,1.000000,0.283654,0.126820,0.085937,0.071010,0.085072,0.141942,0.055565,...,0.046793,0.123649,0.043285,0.015861,0.041156,0.022289,0.036729,0.131806,0.053766,0.012281
add volume,0.130217,0.503426,0.283654,1.000000,0.083987,0.084393,0.194558,0.088197,0.206515,0.067119,...,0.053727,0.109287,0.055342,0.017984,0.045149,0.027820,0.033971,0.106977,0.079740,0.033707
append,0.098375,0.074089,0.126820,0.083987,1.000000,0.069418,0.080701,0.178371,0.124796,0.041695,...,0.063082,0.146287,0.142212,0.178445,0.074746,0.012820,0.031888,0.083919,0.080358,0.105399
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
winsat mfmedia,0.030104,0.033251,0.022289,0.027820,0.012820,0.007020,0.015675,0.011106,0.019471,0.012126,...,0.035893,0.020954,0.009901,0.000000,0.007845,1.000000,0.000000,0.010928,0.017402,0.000000
wmic,0.004136,0.000000,0.036729,0.033971,0.031888,0.041844,0.023888,0.038840,0.009304,0.025364,...,0.035634,0.083271,0.015610,0.000000,0.000000,0.000000,1.000000,0.027183,0.023391,0.000000
writer,0.059803,0.055559,0.131806,0.106977,0.083919,0.107712,0.076426,0.095239,0.106717,0.038606,...,0.047391,0.099907,0.059851,0.000000,0.010124,0.010928,0.027183,1.000000,0.009395,0.000000
wscript,0.030928,0.069300,0.053766,0.079740,0.080358,0.018151,0.058344,0.028010,0.043059,0.037756,...,0.041072,0.005137,0.132321,0.166444,0.047204,0.017402,0.023391,0.009395,1.000000,0.000000


In [16]:
sim_df.loc['del']['erase']

0.9185181048815297

In [ ]:
#sim_df.to_csv('./data/windows/windows_desc_sim_df.csv')
sim_df = pd.read_csv('./data/windows/windows_desc_sim_df.csv', index_col=0)

In [ ]:
sim_df

In [ ]:
sim_df.loc['del']['erase']

In [ ]:
id_ar = np.identity((len(sim_df.columns)))
temp = sim_df.round(3) - id_ar
temp2 = temp[temp.idxmax()]
a = temp.max()
b = temp.idxmax()
c = dict(a)
d = dict(b)
sim_sc = []
for k, v in c.items():
    sim_sc.append([k, d[k], c[k]] )
sim_sc_pd = pd.DataFrame(sim_sc, columns=['first', 'second', 'score'])
asc = sim_sc_pd.sort_values('score', ascending=False)
figure = plt.figure(figsize=(12, 8))
plt.plot(asc['first'], asc['score'])
plt.xticks(rotation=90)
plt.savefig('./results/windows/win_description_max_sim_per_command.png');

In [ ]:
figure = plt.figure(figsize=(8,8))
plt.pcolor(sim_df);

In [ ]:
df_high_sim, new_sim = get_max_sim_comds(sim_df, 0.75)

In [ ]:
df_high_sim

In [ ]:
df_plot = create_matrix_high_sim_comds(df_high_sim)

In [ ]:
df_plot.head()

In [ ]:
score_values = df_plot.to_numpy('float').tolist()

fig, ax = plt.subplots(1,1)
fig.set_figheight(20)
fig.set_figwidth(20)
im = ax.imshow(score_values)
x_axis_values = list(df_plot.columns.values)
y_axis_values = list(df_plot.index.values)
# We want to show all ticks...
ax.set_xticks(np.arange(len(x_axis_values)))
ax.set_yticks(np.arange(len(y_axis_values)))
# ... and label them with the respective list entries
ax.set_xticklabels(x_axis_values)
ax.set_yticklabels(y_axis_values)

# Rotate the tick labels and set their alignment.
plt.setp(ax.get_xticklabels(), rotation=90, ha="right",
         rotation_mode="anchor")
plt.setp(ax.get_yticklabels(),rotation=0, ha="right",
         rotation_mode="anchor")


ax.set_title("")
#plt.savefig('./results/windows/wind_desc_with_high_sim.png')
plt.show()

In [ ]:
G = nx.Graph()
for key, value in new_sim.items():
    for v in value:
        G.add_edge(key,v)

        
figure = plt.figure(figsize=(20,16))
colors = ['r','b','y','c']*39
options = {
    'node_color': 'pink',
    'node_size': 300}
pos = nx.spring_layout(G)
nx.draw(G, with_labels=True,pos=pos,edge_color=colors, **options)

In [ ]:
comm1_unique = list((df_high_sim['comm1'].unique()))
comm2_unique = list((df_high_sim['comm2'].unique()))
for cmd in comm2_unique:
    if cmd not in comm1_unique:
        comm1_unique.append(cmd)
new = sim_df.loc[comm1_unique]

In [ ]:
fig = ff.create_dendrogram(new,  labels=new.index)
fig.update_layout(width=1600, height=800)
fig.write_image('./results/windows/wind_desc_dendo_high_sim.png')
fig.show()

In [ ]:
subgraphs =  list(nx.connected_components(G))
new = dict()
for i in range(len(subgraphs)):
    new[next(iter(subgraphs[i]))] = len(subgraphs[i])
new = {k: v for k, v in sorted(new.items(), key=lambda item: item[1],reverse=True)}    

In [ ]:
subgraphs

In [ ]:
new

In [ ]:
new['auditpol *'] = new.pop('auditpol list')
new['ftp get, put, recv, send'] = new.pop('ftp recv')
new['wbadmin *'] = new.pop('wbadmin start recovery')
new['convert *'] = new.pop('convert mbr')
new['create volume *'] = new.pop('create volume simple')
new['select *'] = new.pop('select disk')
new['ftp ls, dir'] = new.pop('ftp mls')
new['logman update *'] = new.pop('logman update counter')
new['bitsadmin *'] = new.pop('bitsadmin getbytestransferred')

new

In [ ]:
new = {k: v for k, v in sorted(new.items(), key=lambda item: item[1],reverse=True)} 

In [ ]:
new = {key:val for key, val in new.items() if val >2}

In [ ]:
import seaborn as sns
sns.set()
fig= plt.figure(figsize=(12,6))
x = list(new.keys())
y = list(new.values())
plt.bar(x, y,  color='lightgreen')
plt.xticks()

plt.yticks(rotation='horizontal')
plt.title('Clusters and their sizes')
for i in range(len(x)):
    if i>2:
         plt.annotate(str(x[i]), xy=(x[i],y[i]), ha='center', va='center', rotation=90, fontsize=24)
    else:
        plt.annotate(str(x[i]), xy=(x[i],y[i]), ha='center', va='top', rotation=90, fontsize=24)

plt.xticks([])
plt.tight_layout()
plt.savefig('./results/windows/windows_desc_histogram.png')
